In [4]:
from ultralytics import YOLO
import cv2 as cv
import os

model = YOLO('/home/kerdizheng/Desktop/icevision/icevision_v1/train3/weights/best.onnx')

plots_path = '/home/kerdizheng/Desktop/icevision/temp'

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [5]:
def run_model(img_path):
    results = model(img_path, imgsz=800)

    boxes = results[0].boxes.data.cpu().tolist()
    labels = results[0].boxes.cls.cpu().tolist()

    print(boxes)
    print(labels)

    img = cv.imread(img_path)

    for label, box in zip(labels, boxes):
        cv.rectangle(
            img, 
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color=(0,0,255),
            thickness=1
        )

    
    cv.imwrite(os.path.join(plots_path, os.path.basename(img_path)), img)

In [6]:
run_model("/home/kerdizheng/Desktop/icevision/data/valid/images/video3_frame0.png")

Loading /home/kerdizheng/Desktop/icevision/icevision_v1/train3/weights/best.onnx for ONNX Runtime inference...
WARNING ⚠️ Failed to start ONNX Runtime with CUDA. Using CPU...
Using ONNX Runtime CPUExecutionProvider

image 1/1 /home/kerdizheng/Desktop/icevision/data/valid/images/video3_frame0.png: 864x864 9 players, 356.2ms
Speed: 6.8ms preprocess, 356.2ms inference, 0.7ms postprocess per image at shape (1, 3, 864, 864)
[[193.4604949951172, 122.71208190917969, 250.71824645996094, 202.7640838623047, 0.7767724990844727, 0.0], [504.31298828125, 187.7971954345703, 573.2272338867188, 267.84027099609375, 0.7256882786750793, 0.0], [590.8347778320312, 272.9807434082031, 654.4566040039062, 384.3087463378906, 0.6840597987174988, 0.0], [21.745771408081055, 107.25154876708984, 83.22175598144531, 182.4069061279297, 0.6788039207458496, 0.0], [678.2392578125, 339.6797180175781, 750.2464599609375, 440.67388916015625, 0.6347365379333496, 0.0], [562.5900268554688, 193.01806640625, 605.2793579101562, 275.

In [ ]:
# base = '/home/ubuntu/YOLO/lngxa_unit 71_rev.b_combined_pg4/lngxa_unit 71_rev.b_combined_pg4_200_inference/input'

# for img in os.listdir(base):
#     img_path = os.path.join(base, img)

#     run_model(img_path)

In [ ]:
import onnxruntime as ort
import numpy as np
import cv2 as cv
import torch

# Load the ONNX model
session = ort.InferenceSession('/home/ubuntu/YOLO/projects/Signals_Iter09_v10/train/weights/best.onnx')

img_path = '/home/ubuntu/YOLO/scripts/iENG-PID/temp/lngxa_unit 71_rev.b_combined_pg4/Shape_ComplexShapesYOLOv8Processor_Signals/input/7_3600_0_4400_800.png'

# Create dummy input matching the input size and format of your model
# dummy_input = np.random.randn(1, 3, 800, 800).astype(np.float32)  # Adjust size as necessary
img = cv.imread(img_path)

img = np.transpose(img, (2, 0, 1))

# Run inference
outputs = session.run(None, {'images': [img]})

print(outputs[0])


In [ ]:
outputs[0].shape


In [ ]:

boxes = []

for item in outputs[0]:
    for prediction in item:
        # print(prediction)
        x, y, w, h, conf, ID = prediction
        print(x, y, w, h, conf, ID)
        if conf >= 0.9:
            boxes.append([ID, x, y, w, h])

# print(boxes)

In [ ]:
img = cv.imread(img_path)
for box in boxes:
    cv.rectangle(
        img,
        (int(box[1]), int(box[2])),
        (int(box[3]), int(box[4])),
        color=(0,0,255),
        thickness=1
    )

cv.imwrite('/home/ubuntu/YOLO/temp_plots/detections.png', img)

# Plotting

In [ ]:
import os
import json
import cv2 as cv
from shutil import rmtree

base = '/home/ubuntu/YOLO/scripts/iENG-PID/temp'
moduleInstanceName = "Shape_ComplexShapesYOLOv8Processor_Signals"

plots_path = '/home/ubuntu/YOLO/temp_plots'

if os.path.exists(plots_path):
    rmtree(plots_path)
    
os.makedirs(plots_path)

for file in os.listdir(base):
    img_path = os.path.join(base, file, 'graphicsMask.png')
    json_path = os.path.join(base, file, f"{file}.json")
    
    img = cv.imread(img_path)
    
    with open(json_path, 'r') as f:
        json_data = json.load(f)

    for annot in json_data[moduleInstanceName]:
        MinX = annot['MinX']
        MinY = annot['MinY']
        MaxX = annot['MaxX']
        MaxY = annot['MaxY']
        # label = length_corrected_annot['Value']
        
        cv.rectangle(
            img,
            (MinX, MinY),
            (MaxX, MaxY),
            color = (0,0,255),
            thickness=2
        )
    
    cv.imwrite(os.path.join(plots_path, f"{file}.png"), img)
        